## Motion control

### Motion function and thymio connection/deconnection

In [6]:
#Import
import time
import asyncio
from tdmclient import aw, ClientAsync
import math
import numpy as np

# conversion thymio speed to mm/s
Thymio_to_mms = 0.349
px_to_mm = 140/100
#Thymio_to_pxs = Thymio_to_mms * mm_to_px 

# Thymio connection
async def connect_Thymio():
    """
    Establish a connection with the Thymio if possible
    """
    global node, client
    try:
        client = ClientAsync()
        node = await asyncio.wait_for(client.wait_for_node(), timeout=2.0)
        await node.lock()
        print("Thymio connected")

    except asyncio.TimeoutError:
        print("Thymio not connected: Timeout while waiting for node.")
    except Exception as e:
        print(f"Thymio not connected: {str(e)}")
        
# Thymio disconnection
def disconnect_Thymio():
    """
    Enable to disconnect the Thymio
    """
    aw(node.stop())
    aw(node.unlock())
    print("Thymio disconnected")
        
# Thymio control motor speeds  
async def set_speeds(left_speed, right_speed):
    """
    Enable to set the speed of the Thymio's wheels
    """
    global node
    v = {
        "motor.left.target":  [left_speed],
        "motor.right.target": [right_speed],
    }
    await node.set_variables(v)
    
async def motors_stop():
    """
    Stop the Thymio
    """
    global node
    v = {
        "motor.left.target":  [0],
        "motor.right.target": [0],
    }
    await node.set_variables(v)    
    
# Check
#await connect_Thymio()
#await set_speeds(40, -40)
#time.sleep(2)
#await motors_stop()
#disconnect_Thymio()

# Turn a specified angle 

# Constants
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)

async def turn(angle):
    # Calculate the time needed to turn through the required angle
    rotation_time = (abs(angle) / (2*np.pi)) * TIME_FULL_TURN

    # Turn robot on itself
    # Check the sign of angle
    if np.sign(angle) > 0:
        # If angle is positive, turn in one direction
        await set_speeds(ROTATION_SPEED, -ROTATION_SPEED)
    else:
        # If angle is negative, turn in the other direction
        await set_speeds(-ROTATION_SPEED, ROTATION_SPEED)

    # Wait required time
    time.sleep(rotation_time)

    # Stop the robot
    #await motors_stop()

# Check
#gamma = np.pi
#await connect_Thymio()
#await turn(-gamma)
#disconnect_Thymio()

# Constants
FORWARD_SPEED = 200  # Base speed
TIME_PER_MM = 15.5/1000  # Time it takes for the robot to travel one meter at base speed

async def move_forward(distance_px):
    # Calculate the time needed to travel the requested distance
    
    distance_mm = distance_px * px_to_mm
    travel_time = (distance_mm) * TIME_PER_MM

    # Robot moves forward
    await set_speeds(FORWARD_SPEED, FORWARD_SPEED)

    # Wait for the necessary time
    time.sleep(travel_time)

    # Stop the robot
    #await motors_stop()

# Using the function

#await connect_Thymio()
# distance = 1  # Distance to travel in meters
#await move_forward(distance)
#disconnect_Thymio()

In [ ]:
# Test

import math
import time
import asyncio
import numpy as np
from tdmclient import aw, ClientAsync

reduction_coeff = 20 

robot_vector = [5*reduction_coeff, 10*reduction_coeff, 0]
start = reduction_coeff*[5, 10]
goal = reduction_coeff*[10, 14]

path = np.array([[5, 10], 
                [6,11],
                [7, 12],
                [8,13],
                [9,14],
                [10,15],
                [9,14]])
path = reduction_coeff * path

gamma = 0 # angle entre orientation actuelle et orientation pour le point suivant
current_node = 1 # indice de l'étape suivante
dist_to_node = 0 # distance en pixels séparant le robot de l'étape suivante  
norm_vector_node = np.array([0,0]) # vecteur normalisé reliant la position à la prochaine étape

ANGLE_THRESHOLD = 0.1 # valeur à partir de laquelle le robot arrête de tourner et commence à avancer vers la prochaine étape
FORWARD_THRESHOLD = 1 # valeur à partir de laquelle le robot change d'étape
ROTATION_TIME_THRESHOLD = 1.2
FORWARD_TIME_THRESHOLD = 100

# translation
#forward_duration = 1
#run_speed=118
#run_offset=0

# rotation
#rot_duration = 1.2
#rot_speed = 125

await connect_Thymio()

while True:
    
    # Normal vector
    norm_vector_node[0] = path[current_node][0] - path[current_node-1][0]
    norm_vector_node[1] = path[current_node][1] - path[current_node-1][1] 
    norm_vector_node = norm_vector_node / np.linalg.norm(norm_vector_node)
    
    # angle gamma
    gamma = math.atan2(norm_vector_node[0], norm_vector_node[1]) - robot_vector[2] #angle en rad

    # distance d
    path_current_node = np.array(path[current_node][:])
    path_previous_node = np.array(path[current_node-1][:])
    d = np.linalg.norm(path_current_node - path_previous_node) # distance en px/20

    print(d)
    
    if(abs(gamma) > ANGLE_THRESHOLD):
        await turn(gamma)
    
    robot_vector[2] = gamma + robot_vector[2]
    
    if( d > FORWARD_THRESHOLD):
        await move_forward(d)
    
    current_node = current_node + 1
    
    if current_node == path.shape[0]:
        break

await motors_stop()

In [11]:
async def reach_next_node(current_node, mode, estimated_pos):

    dist_to_node = 0 # distance en pixels séparant le robot de l'étape suivante  
    norm_vector_node = np.array([0,0]) # vecteur normalisé reliant la position à la prochaine étape



    # Normal vector
    norm_vector_node[0] = path[current_node][0] - estimated_pos[0]
    norm_vector_node[1] = path[current_node][1] - estimated_pos[1] 
    norm_vector_node = norm_vector_node / np.linalg.norm(norm_vector_node)
    
    # angle gamma
    gamma = math.atan2(norm_vector_node[0], norm_vector_node[1]) - estimated_pos[2] #angle en rad

    # distance d
    path_current_node = np.array(path[current_node][:])
    path_previous_node = np.array([estimated_pos[0], estimated_pos[1]])
    d = np.linalg.norm(path_current_node - path_previous_node) # distance en px/20

    print(d)
    if(not mode):
        if(abs(gamma) > ANGLE_THRESHOLD):
            await turn(gamma)
    
        #robot_vector[2] = gamma + estimated_pos[2]
        
    if (mode):
        if( d > FORWARD_THRESHOLD):
            await move_forward(d)


Motion control with Kalman:
- form initial position: turn
- check orientation with kalman
- move forward 
- check position with kalman
- from estimated position: turn
- check orientation with kalman
- move forward 
- check position with kalman
- from estimated position: turn
- ...

# Kalman Filter

The localization of the Thymio robot is performed using a Kalman filter. This filtering method is well suited to estimating the position and orientation of a mobile robot from noisy or incomplete measurements. The design of the filter in this project is based on using the position ($x, y$) and orientation ($\theta$) provided by the camera as measurements. In addition, the speed of the robot, provided by the wheel speed sensors ($v_r, v_l$), is used as a prediction. In short, the Kalman filter merges a prediction of the system's future state with a measurement of that state to estimate position probabilistically.

## State-space model

### Prediction

To estimate the robot's future position, a state-space model needs to be developed: 

$$\hat{s}_{a\_priori}^{t+1} = A \cdot \hat{s}_{a\_posteriori}^{t} + B \cdot u^{t} + q^t$$

The prediction of the future state is referred to as $\hat{s}_{a\_priori}^{t+1}$, i.e. the a priori estimate at time t+1. Since the state of the system is defined by its position ($x, y$) and orientation ($\theta$), this gives: 

$$\hat{s}_{a\_priori}^{t+1} = \begin{pmatrix}
\hat{x}_{a\_priori}^{t+1} \\\\
\hat{y}_{a\_priori}^{t+1} \\\\
\hat{\theta}_{a\_priori}^{t+1}
\end{pmatrix}$$

The current state corresponds to the term $\hat{s}_{a\_posteriori}^{t}$, which is the a posteriori estimate at time t. In the same way as above, this gives:

$$\hat{s}_{a\_posteriori}^{t} = \begin{pmatrix}
\hat{x}_{a\_posteriori}^{t} \\\\
\hat{y}_{a\_posteriori}^{t} \\\\
\hat{\theta}_{a\_posteriori}^{t}
\end{pmatrix}$$

The system input at time t is represented by the vector $u^{t}$. This is made up of two terms: translational speed ($v$) and rotational speed ($\omega$). 

$$u^t = \begin{pmatrix}
v \\\\
\omega
\end{pmatrix} $$

These are defined on the basis of the speeds measured by the wheel speed sensors, i.e. the right ($v_r$) and left ($v_l$) speeds, and the spacing between the two wheels ($e$).

$$ v = \cfrac{v_r + v_l}{2} \qquad\qquad \omega = \cfrac{v_r-v_l}{e} $$ 

Matrix A characterizes the evolution of the system state, while matrix B describes the impact of the input on the future state. An odometry-based approach allows us to determine these two matrices by considering a very short time interval ($\delta t$). During this time interval, the robot rotates by $\delta \theta = \omega \cdot \delta t$. Knowing this, and referring to the diagram below, the following system of equations can be established: 

$$\begin{equation}
\begin{cases}
\hat{x}_{a\_priori}^{t+1} = \hat{x}_{a\_posteriori}^{t} + v \cdot \cos\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t \\
\hat{y}_{a\_priori}^{t+1} = \hat{y}_{a\_posteriori}^{t} + v \cdot \sin\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t \\
\hat{\theta}_{a\_priori}^{t+1} = \hat{\theta}_{a\_posteriori}^{t} + \omega \cdot \delta t
\end{cases}
\end{equation}$$

![state-space_model](Images/schematics.png)

The matrix form of this system therefore becomes:

$$\begin{equation}
A = \begin{bmatrix} 
1 & 0 & 0\\ 
0 & 1 & 0 \\ 
0 & 0 & 1
\end{bmatrix}
\qquad\qquad
B = \begin{bmatrix} 
\cos\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t & 0\\
\sin\left(\hat{\theta}_{a\_posteriori}^{t} + \delta \theta^t \right) \cdot \delta t & 0 \\
0 & \delta t 
\end{bmatrix}
\end{equation}$$

The final term $q^t$ of this state-space model represents the stochastic perturbation of the state with covariance matrix Q defined as follows:

$$
Q = \begin{bmatrix} 
q_1 & 0 & 0\\ 
0 & q_2 & 0 \\ 
0 & 0 & q_3
\end{bmatrix}
$$

These diagonal coefficients can be evaluated using an approach similar to that used in Exercise 8 of the MICRO-452 course.  

### Measurement

Having explored the prediction phase of the state-space model, attention now turns to the second essential part: updating the measurements. This stage aims to refine the predictions by integrating real information captured by the camera. The formula governing this step is :

$$ m^{t+1} = C \cdot s^{t+1} + r^{t+1}$$ 

Measurements taken at time t+1 are represented here by the term $m_{t+1}$. The data collected by the camera are therefore:

$$m^{t+1} = \begin{pmatrix}
x_{captured}^{t+1} \\\\
y_{captured}^{t+1} \\\\
\theta_{captured}^{t+1}
\end{pmatrix}$$

The robot's position ($x, y$) and orientation ($\theta$) measured by the camera are used directly as system outputs, without any transformation. The matrix C linking the measurements to the state is therefore defined as follows:

$$C = \begin{bmatrix} 
1 & 0 & 0\\ 
0 & 1 & 0 \\ 
0 & 0 & 1
\end{bmatrix}$$

The term $s^{t+1}$ simply represents the state of the system at time t+1:

$$s^{t+1} = \begin{pmatrix}
x^{t+1} \\\\
y^{t+1} \\\\
\theta^{t+1}
\end{pmatrix}$$

Finally, the last term $r^{t+1}$ of this equation represents noise on measurements with a covariance matrix R defined as follows:

$$
R = \begin{bmatrix} 
r_1 & 0 & 0\\ 
0 & r_2 & 0 \\ 
0 & 0 & r_3
\end{bmatrix}
$$

Note: When the camera's view is obstructed, estimation is only possible using the prediction model.

## Initialization

### Imports

In [12]:
import math
import time
import scipy
import asyncio
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import randrange
from tdmclient import aw, ClientAsync

### Basic functions

### Global variables

In [13]:
# General variables:
update_time = 0.05               # Time in [s] before the next update
thymio_speed_to_mms = 0.4348     # Ratio to convert Thymio speed into mm/s
acquire_data = True              # Boolean that determines whether the program should collect data or not
samples_acquired = 0             # Number of samples acquired
nb_samples = 10                  # Number of samples to reach
camera_on = False                # Boolean that determines whether the camera has vision or not

#Thymio variables: 
robot_diameter = 94              # Distance between the two wheels
v_r = 0                          # Speed of the right wheel 
v_l = 0                          # Speed of the left wheel
v = 0                            # Average translation speed
w = 0                            # Average rotation speed
delta_theta = 0                  # Angle variation
thymio_data = []

#Thymio position and orientation
x0_vision = 10
y0_vision = 10
theta0_vision = np.pi/2
x_vision = 20
y_vision = 20
theta_vision = np.pi

### Filter initialization

In [ ]:
def filter_initialization():
    """
    Initialize the various vectors and matrices requiered for filtering
    
    param vision_x_position: robot x position deduced from the camera vision
    param vision_y_position: robot y position deduced from the camera vision
    param vision_theta_orientation: robot theta orientation deduced from the camera vision
    """
    
    global s_prev_est_a_posteriori, P_prev_est_a_posteriori, A, B, u, C, Q, R

    ## Previous State A Posteriori Estimation Vector
    # Vector representing the estimated state of the system at the previous time step
    s_prev_est_a_posteriori = robot_vector
    #print("\n".join([f"s_prev_est_a_posteriori = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in s_prev_est_a_posteriori]), "]"]))
    
    ## Previous State A Posteriori Covariance Matrix
    # Matrix representing the estimated precision of the previous estimated state
    P_prev_est_a_posteriori = np.array([[1000, 0, 0], 
                                        [0, 1000, 0], 
                                        [0, 0, 1000]]) 
    #print("\n".join([f"P_prev_est_a_posteriori = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in P_prev_est_a_posteriori]), "]"]))

    ## State Matrix
    # Matrix defining how the system evolves from one time step to the next
    A = np.array([[1, 0, 0], 
                  [0, 1, 0], 
                  [0, 0, 1]])
    #print("\n".join([f"A = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in A]), "]"]))
    
    ## Input Matrix 
    # Matrix describing the impact of the input on the state
    B = np.array([[1, 0], 
                  [0, 1], 
                  [0, 0]]); 
    #print("\n".join([f"B = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in B]), "]"]))
    
    ## Input Vector
    # Vector representing control inputs applied to the system 
    u = np.array([[0], 
                  [0]])
    #print("\n".join([f"u = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in u]), "]"]))

    ## Output Matrix
    # Matrix linking measurements to state
    C = np.array([[1, 0, 0], 
                  [0, 1, 0], 
                  [0, 0, 1]])
    #print("\n".join([f"C = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in C]), "]"]))
    
    ## Process Noise Covariance Matrix
    # Covariance matrix representing uncertainty in system dynamics
    Q = np.array([[1, 0, 0], 
                  [0, 1, 0], 
                  [0, 0, 0.1]])
    #print("\n".join([f"Q = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in Q]), "]"]))

    ## Measurement Noise Covariance Matrix
    # Matrix representing uncertainty of camera measurements
    R = np.array([[0.1, 0, 0], 
                  [0, 0.1, 0], 
                  [0, 0, 0.01]])
    #print("\n".join([f"R = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in R]), "]"]))

# Check
#filter_initialization()

## Update

### Updating input vector and matrix

In [ ]:
def update_input(v_l,v_r):
    """
    Update the input vector and matrix
    """
    
    global B,u
    
    print("v_r :", v_r)
    print("v_l :", v_l)
    
    # Average translational speed
    v = (v_r +v_l)/2 
    print("v : ", v)
    
    # Average rotational speed
    w = (v_r -v_l)/robot_diameter 
    
    # Input vector
    u = np.array([[v], 
                  [w]]) 
    #print("\n".join([f"u = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in u]), "]"]))
    
    # Angle variation
    delta_theta = w * update_time
    
    # Input matrix
    B = np.array([[np.cos(delta_theta + s_prev_est_a_posteriori[2][0])*update_time, 0],
                  [np.sin(delta_theta + s_prev_est_a_posteriori[2][0])*update_time, 0], 
                  [0, update_time]]); 
    #print("\n".join([f"B = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in B]), "]"]))

# Check
#update_input()

## Filtering

In [ ]:
def kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori):
    """
    Estimates the current state using the input sensor data and the previous state
    
    param s_prev_est_a_posteriori: previous state a posteriori estimation
    param P_prev_est_a_posteriori: previous state a posteriori covariance
    
    return s_est_a_posteriori: new a posteriori state estimation
    return P_est_a_posteriori: new a posteriori state covariance
    """
    
    ## Prediciton through the a priori estimate
    # estimated mean of the state
    s_est_a_priori = np.dot(A, s_prev_est_a_posteriori)+ np.dot(B, u);
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_prev_est_a_posteriori, A.T)) + Q
    
    ## Update         
    # m, C, and R for a posteriori estimate, depending on the detection of the camera
    if camera_on == True:
        m = np.array([[x_vision], 
                      [y_vision], 
                      [theta_vision]])
        # innovation / measurement residual
        i = m - np.dot(C, s_est_a_priori);
        # measurement prediction covariance
        S = np.dot(C, np.dot(P_est_a_priori, C.T)) + R;     
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(C.T, np.linalg.inv(S)));
        # a posteriori estimate
        s_est_a_posteriori = s_est_a_priori + np.dot(K,i);
        P_est_a_posteriori = P_est_a_priori - np.dot(K,np.dot(C, P_est_a_priori));
    else:
        K = 0 # Kalman gain is null because the camera can't deliver any data
        # a posteriori estimate
        s_est_a_posteriori = s_est_a_priori;
        P_est_a_posteriori = P_est_a_priori;
     
    return s_est_a_posteriori, P_est_a_posteriori

#Check
#kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori)

## Testing

In [ ]:
await connect_Thymio()

speed_test_value = randrange(200)      
    
node.send_set_variables(set_speeds(speed_test_value, speed_test_value))

time.sleep(0.5)

# Test

# Initialization
filter_initialization()

# Initial state
print("\n".join([f"Initial state = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in s_prev_est_a_posteriori]), "]"]))

for _ in range(3):
    await get_speeds()         # Get the speeds
    update_input()             # Update inputs
    speeds_graph()             # Plot the graph of the speeds
    s_prev_est_a_posteriori, P_prev_est_a_posteriori = kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori)
    print("\n".join([f"Next state = [", "\n".join([" , ".join([f"{x: 10.2f}" for x in row]) for row in s_prev_est_a_posteriori]), "]"]))

node.send_set_variables(set_speeds(0, 0))

disconnect_Thymio()

What need to be done with the Thymio used for the final demo:
- Evaluate the Q and R matrices
- Find the speed in pixels per second : $v_{px/s} = v_{thymio} \cdot thymio\_speed\_to\_mms \cdot mm\_to\_px$
- Time in the B matrix: update_time or 1.5*((nb_samples)*update_time)? Depends on how we decide to get the speed... 

## 5 Local Navigation
<a id="local-navigation"></a>

### Functions

## 7 Main
<a id="main"></a>

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interactive
from matplotlib import colors
from PIL import Image
import math
import time


ANGLE_THRESHOLD = 0.1 # valeur à partir de laquelle le robot arrête de tourner et commence à avancer vers la prochaine étape
FORWARD_THRESHOLD = 1 # valeur à partir de laquelle le robot change d'étape
ROTATION_TIME_THRESHOLD = 1.2
FORWARD_TIME_THRESHOLD = 100
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)
FORWARD_SPEED = 200  # Base speed
TIME_PER_MM = 15.5/1000  # Time it takes for the robot to travel one meter at base speed

In [10]:
await connect_Thymio()

current_node = 0
filter_initialization()

while True:
    
    reach_next_node(current_node, 0, s_prev_est_a_posteriori)
    update_input(ROTATION_SPEED, -ROTATION_SPEED)
    s_prev_est_a_posteriori, P_prev_est_a_posteriori = kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori)
    
    reach_next_node(current_node, 1, s_prev_est_a_posteriori)
    update_input(FORWARD_SPEED, FORWARD_SPEED)
    s_prev_est_a_posteriori, P_prev_est_a_posteriori = kalman_filter(s_prev_est_a_posteriori, P_prev_est_a_posteriori)
    
    current_node = current_node + 1
    
    if (current_node == path.shape()):
        break
        
        

await disconnect_Thymio()
        
    
    
    


Thymio not connected: Timeout while waiting for node.


NameError: name 'robot_vector' is not defined

## 6 Conclusion
<a id="conclusion"></a>